### Starvation and Preemption
饥饿和抢占是一组相关的概念, 最好放在一起讨论. 如下的fairshare均指Instantaneous fairshare

#### 一. enable preemption    
fairScheduler 会抢占一些队列中的container, 如果该队列的资源使用超过了队列的Instantaneous FairShare, 且有如下两个条件发生: 
* 存在饥饿状态的应用
* preemption是enabled状态

集群在什么时候Preemption是enabled的? 档如下两个条件同时发生时:
* `yarn.scheduler.fair.preemption`参数设置为true
* cluster级的资源利用率超过了一个阈值, 默认是80%, 由参数`yarn.scheduler.fair.premption.cluster-utilization-threshold`设置  

因为在cluster级的资源利用率很低时对container执行preemption的话会影响集群效率, cluster级的资源利用率是根据cpu core和内存使用量计算得来的.  
当然, 队列也可以被设置为不可被抢占的, 为了让该队列会有较高的优先级

#### 二. fairshare starvation
一个applicaiton时fareshare starved的, 如果以下条件全部满足: 
* 该applicaiton有未满足的资源
* 该application的资源占有量低于其Instantaneous fairshare
* 该application占用的资源, 小于1个阈值* 它的Instataneous fairshare, 且超过了fairshare preemption timeout  
  该阈值默认为0.5, 而fairshare preemption timeout默认未被设置, 因此需要显式的被设置才能让一个应用成为饥饿的
  
&nbsp;&nbsp;&nbsp;&nbsp;值得注意的是, 开启preemption也不能保证1个application或queue可以得到其所有的Instantaneous fairshare, 只是说在fairshare preemption timeout之后, application的资源占用可以达到`阈值*其Instantaneous fairshare`   
&nbsp;&nbsp;&nbsp;&nbsp;如果想把一个queue的优先级提高, 可以把该queue的starvation阈值调的很高, stavation timeout的时间调的很低, 且设置为不可被抢占的  
&nbsp;&nbsp;&nbsp;&nbsp;Instantaneous fairshare是queue级别的, 而fairshare starvation是application级别的. 这很重要, 因为可能一个队列并不是饥饿的, 但处在其中的applicaiton却是饥饿的. 一个占用资源超过其Instantaneous fairshare的application, 其container可以被任何队列的starved application占用  

#### 三. minshare starvation
一个queue是minshare starvation的, 只要如下条件都满足 : 
* 队列中存在为满足资源的application
* 队列的资源占用率小于其minshare
* 队列的资源占用率小于其minshare, 持续了minshare preemption timeout的时间    
  该超时时间默认未被设置, 因此如果想要让queue可能被认为是minshare starvation的, 就需要显示的被设置  
  
minshare starvation是对列级别的, 一个application不可能处于minshare starvation状态

#### 四. Containers to preemption
当有多个节点同时满足被抢占的资源请求时, 优先选择在container数量最少的节点抢占. 一个container时可被抢占的, 如果以下两个条件同时满足 : 
* 该container所属的queue被设置成可被抢占的
* 杀死该container后, 不会引起对应的application占用资源小于其fairshare  

minshare并不用于决定哪个container被占用, 它只用于配置是否是starvation的. 这很重要, 值得反复重复, 无论fairshare和minshare starvation是否配置了, 只有超过其fairshare的application可被抢占. 如下表定义: 
<img src="img/preemption.png">

#### 五. 配置starvation的最佳实践
配置fairshare starvation是被推荐的, 而不推荐配置minshare starvation. 因为这会引起额外的复杂度 : 
* 当queue发生minshare starvation, 队列中的container超过了minshare但没有超过fairshare的不会被抢占
* 同一个队列中的application, fairshare-starved是相同的
* 确保所有的minimum resources加起来后小于整体资源